<a href="https://colab.research.google.com/github/datasciapps/trifels2025/blob/master/binn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge-enhanced biomarker discovery workshop

a.k.a. Embedding prior knowledge into multi-omics integration.

[**AI in Bioinformatics Spring School**](https://trifels2025.kl.dfki.de/)  
_Annweiler-am-Triefels, Rhineland-Palatinate, Germany, March 2025_

## Learning outcomes

1. Compare approaches for integrating biological knowledge into biomarker discovery
2. Design and critique deep learning architectures for multi-omics tasks
3. Get hands-on experience with 'BINN'/'VNN' frameworks and identify practical challenges

## Resources

1. _This_ Colab notebook ([link](https://colab.research.google.com/drive/1pqFEGiqWA0Ku6SovHSDUoLCE3uVKgGlm?usp=sharing))
2. Open-source software packages --- *see Table, below*
3. Cancer Genome Atlas data --- <https://www.cbioportal.org>
4. Workshop slides [ADD LINK]

There are a handful for tools for fitting biologically-informed neural networks in Python. Most are designed for single-omic inputs.

Package    | Reference     | Learning task | Modality | Input data type
-----------|------------|---------------|------------|------------
[BINN](https://infectionmedicineproteomics.github.io/BINN)       | Hartman et al ([2023](https://www.nature.com/articles/s41467-023-41146-4)) | supervised    | single-omic | proteomics
[VEGA](https://github.com/LucasESBS/vega/tree/main)       | Seninge et al ([2021](https://www.nature.com/articles/s41467-021-26017-0))   | unsupervised  | single-omic | transcriptomics
[GenNet](https://github.com/arnovanhilten/GenNet)     | van Hilten et al ([2021](https://www.nature.com/articles/s42003-021-02622-z)) | supervised    | single-omic | genomics
[GenNet-multi-omic](https://github.com/ArnovanHilten/GenNet-multi-omic) | van Hilten et al ([2024](https://www.nature.com/articles/s41540-024-00405-w)) | supervised | multi-omics | transcriptomics & epigenomics



# Practical task

In this worksheet, the goal is to compare and contrast different methods for multi-omics integration to predict survival of patients with cancer.
We consider the effect of embedding of prior knowledge on the model and its outputs.

## Download and wrangle omics data from the Cancer Genome Atlas (TCGA)

I've done this so you don't have to.
We download data from [cBioPortal](https://www.cbioportal.org/datasets).
The scripts below will bring it directly into the Colab working directory, so you don't have to worry about saving big files onto your laptop, or bringing the internet in Annweiler-am-Triefels to a stuttering halt.

This might take a minute or so:

In [ ]:
study_id = "prad_tcga_pan_can_atlas_2018"                               # Prostate cancer cohort
!wget -nc https://cbioportal-datahub.s3.amazonaws.com/{study_id}.tar.gz # Download to Colab
!tar -xzf {study_id}.tar.gz                                             # Unzip archive
%ls {study_id}                                                          # Show files

File ‘prad_tcga_pan_can_atlas_2018.tar.gz’ already there; not retrieving.

case_lists/
data_armlevel_cna.txt
data_clinical_patient.txt
data_clinical_sample.txt
data_clinical_supp_hypoxia.txt
data_cna_hg19.seg
data_cna.txt
data_gene_panel_matrix.txt*
data_genetic_ancestry.txt
data_log2_cna.txt
data_methylation_hm27_hm450_merged.txt
data_mrna_seq_v2_rsem.txt
data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt
data_mrna_seq_v2_rsem_zscores_ref_diploid_samples.txt
data_mrna_seq_v2_rsem_zscores_ref_normal_samples.txt
data_mutations.txt
data_resource_definition.txt
data_resource_patient.txt
data_rppa.txt
data_rppa_zscores.txt
data_sv.txt
data_timeline_sample_acquisition.txt
data_timeline_status.txt
data_timeline_treatment.txt
LICENSE
meta_armlevel_cna.txt
meta_clinical_patient.txt
meta_clinical_sample.txt
meta_cna_hg19_seg.txt
meta_cna.txt
meta_gene_panel_matrix.txt*
meta_genetic_ancestry.txt
meta_log2_cna.txt
meta_methylation_hm27_hm450_merged.txt
meta_mrna_seq_v2_rsem.txt
meta_mrna_seq_v2_rs

You can have a peek at some of the raw data in the folder using shell commands.

In [ ]:
%cat {study_id}/README.md                                               # (Optional) browse the documentation

## Improving PanCancer Atlas Data: (transformation steps for the new data types being added)

### The Timeline/Treatment data:

**Data Source**
- GDAC Firehose: https://gdac.broadinstitute.org/
- File Used: `Merge_Clinical.Level_1.20160128` (clin.merged.txt) for each cancer type.

**Data Transformation** 
- The detailed transformation steps are listed in the Pull Request [here](https://github.com/cBioPortal/datahub/pull/1597)

### The Genetic Ancestry data:


**Data Source**
- GDAC Firehose: https://gdc.cancer.gov/about-data/publications/CCG-AIM-2020
- File Used: `Admixture_by_sample.txt` (Admix percent by sample) for each cancer type.


### The Methylation data:

**Data Source**
- GDAC Firehose: https://gdc.cancer.gov/node/977
- File Used: `jhu-usc.edu_PANCAN_HumanMethylation450.betaValue_whitelisted.tsv` (DNA methylation 450K only beta value data matrix) for each cancer type.

**Data Transformation** 
 - The detailed transformation steps are listed in the Pull Request [here](https://

In [ ]:
!head -n6 {study_id}/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt  # (Optional) peek at the RNAseq data

Hugo_Symbol	Entrez_Gene_Id	TCGA-2A-A8VL-01	TCGA-2A-A8VO-01	TCGA-2A-A8VT-01	TCGA-2A-A8VV-01	TCGA-2A-A8VX-01	TCGA-2A-A8W1-01	TCGA-2A-A8W3-01	TCGA-2A-AAYF-01	TCGA-2A-AAYO-01	TCGA-2A-AAYU-01	TCGA-4L-AA1F-01	TCGA-CH-5737-01	TCGA-CH-5738-01	TCGA-CH-5739-01	TCGA-CH-5740-01	TCGA-CH-5741-01	TCGA-CH-5743-01	TCGA-CH-5744-01	TCGA-CH-5745-01	TCGA-CH-5746-01	TCGA-CH-5748-01	TCGA-CH-5750-01	TCGA-CH-5751-01	TCGA-CH-5752-01	TCGA-CH-5753-01	TCGA-CH-5754-01	TCGA-CH-5761-01	TCGA-CH-5762-01	TCGA-CH-5763-01	TCGA-CH-5764-01	TCGA-CH-5765-01	TCGA-CH-5766-01	TCGA-CH-5767-01	TCGA-CH-5768-01	TCGA-CH-5769-01	TCGA-CH-5771-01	TCGA-CH-5772-01	TCGA-CH-5788-01	TCGA-CH-5789-01	TCGA-CH-5790-01	TCGA-CH-5791-01	TCGA-CH-5792-01	TCGA-CH-5794-01	TCGA-EJ-5494-01	TCGA-EJ-5495-01	TCGA-EJ-5496-01	TCGA-EJ-5497-01	TCGA-EJ-5498-01	TCGA-EJ-5499-01	TCGA-EJ-5501-01	TCGA-EJ-5502-01	TCGA-EJ-5503-01	TCGA-EJ-5504-01	TCGA-EJ-5505-01	TCGA-EJ-5506-01	TCGA-EJ-5507-01	TCGA-EJ-5508-01	TCGA-EJ-5509-01	TCGA-EJ-5510-01	TCGA-EJ-5511-01	TCGA-EJ-5512-

In [ ]:
!head -n6 {study_id}/data_clinical_patient.txt                          # (Optional) see clinical var definitions

#Patient Identifier	Subtype	TCGA PanCanAtlas Cancer Type Acronym	Other Patient ID	Diagnosis Age	Sex	Neoplasm Disease Stage American Joint Committee on Cancer Code	American Joint Committee on Cancer Publication Version Type	Last Communication Contact from Initial Pathologic Diagnosis Date	Birth from Initial Pathologic Diagnosis Date	Last Alive Less Initial Pathologic Diagnosis Date Calculated Day Value	Ethnicity Category	Form completion date	Neoadjuvant Therapy Type Administered Prior To Resection Text	ICD-10 Classification	International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code	International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code	Informed consent verified	New Neoplasm Event Post Initial Therapy Indicator	American Joint Committee on Cancer Metastasis Stage Code	Neoplasm Disease Lymph Node Stage American Joint Committee on Cancer Code	American Joint Committee on Cancer Tumor Stage Code	Person Neoplasm Cancer Status	Pri

The data from TCGA has some quirks, requiring some wrangling to get it into a 'tidy' format where rows represent patients or samples and columns represent features. Some helper functions will get it into Pandas DataFrames.

In [ ]:
import pandas as pd

def read_clinical_data(study_id):
  clinical = pd.read_table(study_id + '/data_clinical_patient.txt', skiprows=4)
  status_cols = [col for col in clinical.columns if col.endswith('S_STATUS')]
  # Split columns with values like "0:ALIVE OR DEAD TUMOR FREE"
  # into binary 0/1 indicators, while keeping the labels for clarity
  for col in status_cols:
    clinical[[col, col + '_LABEL']] = clinical[col].str.split(':', n=1, expand=True)
    clinical[col] = clinical[col].astype('Int64')
  clinical['Patient_ID'] = clinical['PATIENT_ID'] + '-01' # for linkage
  return clinical.set_index('Patient_ID')

dtypes = {'Hugo_Symbol': 'string', 'Entrez_Gene_Id': 'string'}

def read_cna_data(study_id):
  return pd.read_table(study_id + '/data_cna.txt', dtype=dtypes)

def read_expression_data(study_id):
  return pd.read_table(
      study_id + '/data_mrna_seq_v2_rsem_zscores_ref_all_samples.txt',
      dtype = dtypes)

def transpose_omic_data(df):
  df_t = df.set_index(['Hugo_Symbol', 'Entrez_Gene_Id']).T
  df_t.index.name = 'Patient_ID'
  return df_t

Let's import the data.

In [ ]:
# Get the clinical data, including patient outcomes
clinical = read_clinical_data(study_id)

# Get the transcriptomics (RNAseq) data
mrna = read_expression_data(study_id)
mrna = transpose_omic_data(mrna)

# Get the Copy Number Alterations data
cna = read_cna_data(study_id)
cna = transpose_omic_data(cna)

Check for missing values.

In [ ]:
def preprocess_data(df):
  # Drop columns with missingness
  df = df.dropna(axis=1)
  return df

preprocess_data(mrna)
mrna

Hugo_Symbol          <NA>            UBE2Q2P2 HMGB1P1    <NA>          \
Entrez_Gene_Id  100130426 100133144 100134869   10357   10431  155060   
Patient_ID                                                              
TCGA-2A-A8VL-01   -3.9250    0.9961    0.2808 -0.6698  1.5187  0.4726   
TCGA-2A-A8VO-01   -3.9250    0.8972   -0.0291 -1.5009 -0.0310  0.7544   
TCGA-2A-A8VT-01   -3.9250    2.2028    2.0529 -0.1843 -1.4942  1.7714   
TCGA-2A-A8VV-01   -3.9250    1.0604    0.2002  0.2431  0.1970  0.0339   
TCGA-2A-A8VX-01   -3.9250   -1.4829   -0.6803 -0.1020 -0.1675  0.5177   
...                   ...       ...       ...     ...     ...     ...   
TCGA-QU-A6IL-01   -3.9250    0.7901   -2.1185 -0.7843  0.2891  1.6776   
TCGA-QU-A6IM-01   -3.9250    1.9255   -4.0599 -2.5256  0.5977  0.9925   
TCGA-QU-A6IN-01   -0.2381   -2.8149   -0.0429  0.0777  1.4064  0.6638   
TCGA-QU-A6IO-01   -3.9250   -2.8149   -1.7740 -1.6297  0.8572  1.3956   
TCGA-QU-A6IP-01   -3.9250    0.4277    0.0074  0.8809 -1.2377 -0.1107   

Hugo_Symbol     RNU12-2P   EZHIP  EFCAB8 SRP14P1  ...  ZWILCH   ZWINT    ZXDA  \
Entrez_Gene_Id     26823  340602  388795  390284  ...   55055   11130    7789   
Patient_ID                                        ...                           
TCGA-2A-A8VL-01  -1.8207 -0.7257 -0.7396  0.5098  ... -0.3388 -0.9943 -0.4822   
TCGA-2A-A8VO-01  -1.8207 -0.4969  2.4261 -0.0926  ... -0.5077  0.4074 -0.8093   
TCGA-2A-A8VT-01   0.6468 -0.9740 -0.4385 -1.5329  ... -0.5012 -0.6473  1.6314   
TCGA-2A-A8VV-01  -1.8207  1.2032 -0.8054  0.0868  ... -0.0608 -0.3514 -0.6495   
TCGA-2A-A8VX-01  -1.8207 -1.7339 -0.7559 -0.7974  ...  1.5470  0.8446 -2.1341   
...                  ...     ...     ...     ...  ...     ...     ...     ...   
TCGA-QU-A6IL-01  -1.5181 -1.7339  2.9465 -2.2980  ... -0.7664 -0.2704 -3.0758   
TCGA-QU-A6IM-01   3.3658  2.2315 -1.6605  0.0510  ... -1.4491 -0.5470 -0.5279   
TCGA-QU-A6IN-01   1.1729 -1.7339  0.2855 -2.5453  ... -0.4354 -1.0692 -1.1372   
TCGA-QU-A6IO-01  -1.5181 -1.7339  0.5266 -2.0180  ... -1.0078 -0.8273 -1.0757   
TCGA-QU-A6IP-01  -1.5181  0.2884 -1.6605  1.3419  ... -0.1583  0.2351  0.3339   

Hugo_Symbol        ZXDB    ZXDC  ZYG11A  ZYG11B     ZYX   ZZEF1    ZZZ3  
Entrez_Gene_Id   158586   79364  440590   79699    7791   23140   26009  
Patient_ID                                                               
TCGA-2A-A8VL-01 -1.4010 -0.0668 -0.5152 -0.9247 -0.1804  0.7221 -1.7267  
TCGA-2A-A8VO-01 -0.3129 -1.0530  1.1311 -0.5927  0.1804 -1.0429 -0.8300  
TCGA-2A-A8VT-01  1.3083  2.2305 -0.3449  1.1050 -1.0783  1.7868  2.0489  
TCGA-2A-A8VV-01  0.0283 -0.4782  0.0658 -1.3796  0.4464 -0.4486  1.0313  
TCGA-2A-A8VX-01 -0.5554 -0.5875  1.1058 -0.8375 -0.2113 -1.1054 -0.2218  
...                 ...     ...     ...     ...     ...     ...     ...  
TCGA-QU-A6IL-01 -0.1841 -0.3960 -0.0478 -0.8779 -0.3305 -0.7006  0.9492  
TCGA-QU-A6IM-01 -2.4639 -2.4755 -0.8118 -0.9032  0.5063  0.0355 -0.9174  
TCGA-QU-A6IN-01 -0.3544  1.1911 -0.0200 -0.8794 -0.9727  0.1205  0.6201  
TCGA-QU-A6IO-01 -1.2589  0.0773 -0.2031 -1.0870 -0.0718 -0.6685 -1.7188  
TCGA-QU-A6IP-01  1.2292  0.8662 -1.0269  1.5055 -0.0918  0.6088  0.8540  

[493 rows x 19088 columns]

In [ ]:
import numpy as np

def filter_most_varying_genes(df, threshold=0.8):
  std_devs = df.std()
  cutoff = np.percentile(std_devs, threshold * 100)
  selected_genes = std_devs[std_devs >= cutoff].index
  return selected_genes

mrna.std()

Hugo_Symbol  Entrez_Gene_Id
<NA>         100130426         0.891023
             100133144         1.055346
UBE2Q2P2     100134869         1.031844
HMGB1P1      10357             1.001014
<NA>         10431             1.001014
                                 ...   
ZYG11A       440590            1.001016
ZYG11B       79699             1.001015
ZYX          7791              1.001015
ZZEF1        23140             1.001016
ZZZ3         26009             1.001016
Length: 20531, dtype: float64

In [ ]:
cna_varying_genes

MultiIndex([(          'RHOU',     '58480'),
            (        'ABCB10',     '23456'),
            (         'TTC13',     '79573'),
            (          'ARV1',     '64801'),
            (        'FAM89A',    '375061'),
            (       'MIR1182', '100302132'),
            (        'TRIM67',    '440730'),
            (    'TRIM67-AS1',    '149373'),
            (      'C1orf131',    '128061'),
            (   'TSNAX-DISC1', '100303453'),
            ...
            ('hsa-mir-1302-7',        <NA>),
            (   'hsa-mir-606',        <NA>),
            (   'hsa-mir-607',        <NA>),
            ('hsa-mir-3158-1',        <NA>),
            ('hsa-mir-1244-3',        <NA>),
            (  'hsa-mir-3168',        <NA>),
            (  'hsa-mir-1267',        <NA>),
            (  'hsa-mir-1826',        <NA>),
            (  'hsa-mir-3181',        <NA>),
            (   'hsa-mir-924',        <NA>)],
           names=['Hugo_Symbol', 'Entrez_Gene_Id'], length=5027)

In [ ]:
mrna_varying_genes

MultiIndex([], names=['Hugo_Symbol', 'Entrez_Gene_Id'])

Entrez and Hugo IDs are a little inconsistent in TCGA data. Let's get a mapping from one to another.

We can throw away various features we are not interested in, for example genes that hardly vary among the cohort.

## Standard ML multi-omics integration



In [ ]:
mrna.columns = mrna.columns.droplevel(0)
mrna

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

To integrate the data, we only include patients for whom can link all three data sources.

In [ ]:
merged_data = clinical.join(mrna, how = 'inner')

MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

## Biologically-informed neural networks

A biologically-informed deep learning framework requires three data sources:
1. a dataset of omics from patients (optionally with phenotypes targets, e.g. disease types, or event times),  
i.e. $X$ (and $y$) as for any ordinary ML or statistics task
2. a mapping from observed features to biological concepts, e.g. genes
3. a pathway database or gene regulatory network

In single-omics analyses, the mapping might be natively built into the input dataset, since gene expression data is already labelled according to genes.
However, multi-omics analyses or SNPs will require a mapping from the respective entity, e.g. from the mutation to the corresponding gene.

Some BINN software packages are bundled with a couple of common pathway databases, e.g. Reactome. You can also supply your own topology, if those provided are not enough, if you want to try advanced fusion strategies or if you want to integrate non-omic-type data (e.g. patient demographics) into your model.

## Dependencies

All of the packages above are available on GitHub; you can also install the first two packages from PyPI as follows:

```bash
pip install binn
pip install scvega
```

There are one or two bugs in the visualization module of v0.1.0 of the `binn` package so we recommended installing v0.1.1 or later.

In [ ]:
%pip install binn==0.1.1 #scvega

# Contact

![David Selby](https://dsa.dfki.de/author/david-antony-selby/avatar_hu01a7ec985a1cb8a44b491be532ed5d21_24285_270x270_fill_q75_lanczos_center.jpg)

> **David Selby**  
Data Science and its Applications  
German Research Centre for Artificial Intelligence (DFKI)  
david.selby@dfki.de  
https://dsa.dfki.de  